<a href="https://colab.research.google.com/github/RichardMathewsII/YOLBO/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Open and run this file in Google Colab. Make sure to change runtime type and set hardware accelerator to GPU. The model demo will run more smoothly on the paid Google Colab Pro version, but the free version will run as well.

# Install and Import Dependencies

In [0]:
!git clone https://github.com/RichardMathewsII/YOLBO

In [0]:
cd /content/YOLBO/

In [0]:
!pip install numpy
!pip install keras
!pip install tensorflow
!pip install keras_resnet

In [0]:
!python setup.py build_ext --inplace

In [0]:
%tensorflow_version 1.x

In [0]:
import sys
sys.path.insert(0, '/content/YOLBO')
import cv2
import numpy as np
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu
import os
import tensorflow as tf
import keras
from visualization import *
from object_detection import *
from video_detection import detect_objects_in_video
from look_back import *

# Upload Weights and Sample Video

In [0]:
cd /content/YOLBO/demo

In [0]:
gpu = 0
setup_gpu(gpu)
# download weights ('resnet50_coco_weights.h5') from releases on GitHub repo (https://github.com/RichardMathewsII/YOLBO/releases)
# upload weights and sample video to the demo folder
# check to ensure files have been uploaded
os.chdir('/content/YOLBO/demo')
!ls

# Load RetinaNet Model with Pretrained ResNet50 Backbone

In [0]:
# load retinanet model with pretrained backbone
path = os.getcwd()
model_path = os.path.join(path, 'resnet50_coco_weights.h5')
model = models.load_model(model_path, backbone_name='resnet50')
# NOTE: OSError: Unable to open file (truncated file: eof = 27262976, sblock->base_addr = 0, stored_eof = 152662144)
# This error arises when the weights file was corrupted upon uploading. Delete the file and reupload.

# create object classes dictionary
labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 
                   5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 
                   10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 
                   14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 
                   20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 
                   25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 
                   30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 
                   34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 
                   37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 
                   41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 
                   47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 
                   52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair',
                   57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 
                   61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 
                   66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 
                   70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 
                   75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

# Run the Model on Your Sample Video

The output will appear in the demo folder

In [0]:
# set video path (if using demo video from the releases page, the name is demo_original.mp4)
video_path = '/content/YOLBO/demo/INSERT_VIDEO_FILE_NAME'
video_output = 'demo_annotated.avi'

# for extracting specific frames, set output="frames" and frames=[frame#s] (integers for frame number)
# check video properties and change fps parameter if it is not 30
detect_objects_in_video(video_path, model, labels_to_names, video_output_name, output="video", fps=30, frames=None, yolbo=True)